##### IMPORTANT!!!!! DO NOT SCRAPE IT AGAIN UNLESS THE DATA ARE WRONG

# Web scrapping of Premier League players' statistics from all season

In [21]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq 
import re

# Page to scrap from
my_url = 'https://fbref.com/en/comps/9/history/Premier-League-Seasons'

# Opening up connection, grabbing the page
html = uReq(my_url)
bs = soup(html.read(), 'html.parser')

In [22]:
# Get an url for each PL season details
seasons_links = []
for x in bs.find_all('a', href=re.compile('^\/en/comps/9/[0-9|P]+')):
    pl = bs.find_all('a', text="Premier League")
    fa_pl = bs.find_all('a', text="FA Premier League")
    if x not in pl and x not in fa_pl:
        #print(x.attrs['href'])
        link = 'https://fbref.com' + x.attrs['href']
        print(link)
        seasons_links.append(link)

https://fbref.com/en/comps/9/Premier-League-Stats
https://fbref.com/en/comps/9/10728/2020-2021-Premier-League-Stats
https://fbref.com/en/comps/9/3232/2019-2020-Premier-League-Stats
https://fbref.com/en/comps/9/1889/2018-2019-Premier-League-Stats
https://fbref.com/en/comps/9/1631/2017-2018-Premier-League-Stats
https://fbref.com/en/comps/9/1526/2016-2017-Premier-League-Stats
https://fbref.com/en/comps/9/1467/2015-2016-Premier-League-Stats
https://fbref.com/en/comps/9/733/2014-2015-Premier-League-Stats
https://fbref.com/en/comps/9/669/2013-2014-Premier-League-Stats
https://fbref.com/en/comps/9/602/2012-2013-Premier-League-Stats
https://fbref.com/en/comps/9/534/2011-2012-Premier-League-Stats
https://fbref.com/en/comps/9/467/2010-2011-Premier-League-Stats
https://fbref.com/en/comps/9/400/2009-2010-Premier-League-Stats
https://fbref.com/en/comps/9/338/2008-2009-Premier-League-Stats
https://fbref.com/en/comps/9/282/2007-2008-Premier-League-Stats
https://fbref.com/en/comps/9/229/2006-2007-Prem

In [23]:
# For each season url, get an url to fixtures of that season
fixtures_links = []
for link in seasons_links[0:2]:
    season = uReq(link)
    bs_season = soup(season.read(), 'html.parser')
    fixture = bs_season.find('a', href=re.compile('Premier-League-Scores-and-Fixtures+'))
    fixture_link = 'https://fbref.com' + fixture.attrs['href']
    fixtures_links.append(fixture_link)

In [24]:
# Get the header from table in fixtures
for link in fixtures_links[0]:
    x = 0
    fix = uReq(link)
    bs_fix = soup(fix.read(), 'html.parser')
    div = bs_fix.find('div', {'id':'all_sched'})
    header_row = div.find_all('th', {'class': [' ', 'poptip','sort_default_asc', 'center']})[0:14]
    print(header_row)

ValueError: unknown url type: 'h'

# Get the header from table in fixtures
for link in fixtures_links:
    x = 0
    fix = uReq(link)
    bs_fix = soup(fix.read(), 'html.parser')
    div = bs_fix.find('div', {'id':'all_sched'})
    header_row = div.find_all('th', {'class': [' ', 'poptip','sort_default_asc', 'center']})[0:14]
    print(header_row)
    x = 1
    if x == 1:
        break

In [ ]:
# (continued) Get the header from table in fixtures
header = []
for h in header_row:
    print(h.attrs['data-stat'])
    header.append(h.attrs['data-stat'])

In [ ]:
# Just removing useless cols
header.remove('xg_a')
header.remove('xg_b')

In [ ]:
# Get a table of fixtures for each season
tables = []
for link in fixtures_links:
    fix = uReq(link)
    bs_fix = soup(fix.read(), 'html.parser')
    div = bs_fix.find('div', {'id':'all_sched'})
    tables.append(div)

In [ ]:
# Create a dictionary with data for each statistic (column) by using for loop
data = dict()
for h in header:
    data[h] = []
    if h == 'gameweek':
        for table in tables:
            key = table.find_all('th', {'data-stat': h})
            for k in key:
                if k.get_text() == 'Wk':
                    continue
                elif len(k)==0:
                    data[h].append(0)
                else:
                    data[h].append(k.get_text())
    elif h == 'match_report':
        for table in tables:
            key = table.find_all('td', {'data-stat': h})
            for k in key:
                try:
                    a = k.find('a')
                    link = 'https://fbref.com' + a.attrs['href']
                    data[h].append(link)
                except AttributeError:
                    data[h].append(0)
    else: 
        for table in tables:
            key = table.find_all('td', {'data-stat': h})         
            for k in key:
                if len(k)==0:
                    data[h].append(0)
                else:
                    data[h].append(k.get_text())

for k, v in data.items():
    print(f"{k}: {len(v)}")

In [ ]:
import pandas as pd
 
pl = pd.DataFrame(data=data)
pl.head(50)

In [ ]:
pl.info()

In [ ]:
# Getting rid of empty rows
x = -1
indexes = []
for val in pl['squad_a'].values:
    x += 1
    if val == 0:
        indexes.append(x)
# Check
pl.iloc[indexes].sum()

In [ ]:
pl.drop(indexes, axis=0, inplace=True)

In [ ]:
pl.info()

In [ ]:
# Number of PL matches
first_3_yrs = 3*21*22
onwards = 27*19*20
total_games = first_3_yrs + onwards

print(f"Number of games in dataset equals the real number: {len(pl.score) == total_games}")

## Saving Scrapped Data 

In [ ]:
pl.to_csv('pl_scraped_data/PL_AllMatches_Scores_Links.csv', index=False)

# Getting the data from Fantasy Premier League API
> One disadvantage is that the data there are just for the current season so it is impossible to get historical data
> Resources:
- https://medium.com/@frenzelts/fantasy-premier-league-api-endpoints-a-detailed-guide-acbd5598eb19

In [ ]:
import requests

#### General Endpoints

In [ ]:
# Get API
api = requests.get(' https://fantasy.premierleague.com/api/bootstrap-static/')
json = api.json()

# General "Endpoints"
json.keys()

#### "Fixtures" Endpoint

In [ ]:
# Get API
api = requests.get(' https://fantasy.premierleague.com/api/fixtures/')
json = api.json()

# Number of dictionaries within FIXTURES list
print(f"{len(json)}: number of dictionaries about the fixtures")

game_weeks = []
event_ids = []
for fix in json:
    if fix['started'] == True and fix['finished'] == True:
        game_week = fix['event']
        event_id = fix['id']
        kickoff_time = fix['kickoff_time']
        stats = fix['stats']
        pulse_id = fix['pulse_id']
    game_weeks.append(game_week)
    event_ids.append(event_id)
    
print(event_ids)